## Data Extraction

In [2]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Loading Input Data
input_file = 'Input.xlsx'  # Replace with your actual file name
output_dir = 'Extracted_Articles'
os.makedirs(output_dir, exist_ok=True)

# Reading Input.xlsx using pandas
data = pd.read_excel(input_file)

# Function to extract article content
def extract_article(url, url_id):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract title and text (adjust selectors based on the website structure)
        title = soup.find('h1').get_text(strip=True)
        paragraphs = soup.find_all('p')
        article_text = " ".join([p.get_text(strip=True) for p in paragraphs])
        
        # Save to text file
        file_path = os.path.join(output_dir, f"{url_id}.txt")
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(f"{title}\n\n{article_text}")
        print(f"Article {url_id} extracted successfully.")
        return title, article_text
    except Exception as e:
        print(f"Failed to extract {url_id}: {e}")
        return None, None

# Looping through URLs and extract articles
data['Title'] = ''
data['Article_Text'] = ''

for index, row in data.iterrows():
    url_id = row['URL_ID']
    url = row['URL']
    title, article_text = extract_article(url, url_id)
    data.at[index, 'Title'] = title
    data.at[index, 'Article_Text'] = article_text

# Saving updated input file with titles and article text for reference
data.to_excel('Updated_Input.xlsx', index=False)


Article Netclan20241017 extracted successfully.
Article Netclan20241018 extracted successfully.
Article Netclan20241019 extracted successfully.
Article Netclan20241020 extracted successfully.
Article Netclan20241021 extracted successfully.
Article Netclan20241022 extracted successfully.
Article Netclan20241023 extracted successfully.
Article Netclan20241024 extracted successfully.
Article Netclan20241025 extracted successfully.
Article Netclan20241026 extracted successfully.
Article Netclan20241027 extracted successfully.
Article Netclan20241028 extracted successfully.
Article Netclan20241029 extracted successfully.
Article Netclan20241030 extracted successfully.
Article Netclan20241031 extracted successfully.
Article Netclan20241032 extracted successfully.
Article Netclan20241033 extracted successfully.
Article Netclan20241034 extracted successfully.
Article Netclan20241035 extracted successfully.
Article Netclan20241036 extracted successfully.
Article Netclan20241037 extracted succes

In [3]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

# Load NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Load word lists
with open('positive-words.txt', 'r') as f:
    positive_words = set(f.read().splitlines())

with open('negative-words.txt', 'r') as f:
    negative_words = set(f.read().splitlines())

with open('StopWords_Auditor.txt', 'r') as f:
    stop_words = set(f.read().splitlines())

with open('StopWords_Currencies.txt', 'r') as f:
    stop_words = set(f.read().splitlines())

with open('StopWords_DatesandNumbers.txt', 'r') as f:
    stop_words = set(f.read().splitlines())

with open('StopWords_Generic.txt', 'r') as f:
    stop_words = set(f.read().splitlines())

with open('StopWords_GenericLong.txt', 'r') as f:
    stop_words = set(f.read().splitlines())

with open('StopWords_Geographic.txt', 'r') as f:
    stop_words = set(f.read().splitlines())

with open('StopWords_Names.txt', 'r') as f:
    stop_words = set(f.read().splitlines())
    # Helper Functions
def calculate_scores(text):
    # Tokenize sentences and words
    sentences = sent_tokenize(text)
    words = word_tokenize(text.lower())
    filtered_words = [word for word in words if word.isalpha() and word not in stop_words]

    # Positive, Negative, and Word Count
    positive_score = sum(1 for word in filtered_words if word in positive_words)
    negative_score = sum(1 for word in filtered_words if word in negative_words)
    word_count = len(filtered_words)

    # Polarity and Subjectivity
    polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 1e-6)
    subjectivity_score = (positive_score + negative_score) / (word_count + 1e-6)

    # Average Sentence Length
    avg_sentence_length = word_count / len(sentences)

    # Complex Words and Fog Index
    complex_words = [word for word in filtered_words if count_syllables(word) > 2]
    percentage_complex_words = len(complex_words) / word_count * 100
    fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)
    if len(sentences) == 0:  # Handle edge cases where no sentences exist
        return 0
    avg_sentences=len(filtered_words) / len(sentences)

    # Syllables and Word Length
    syllables_per_word = sum(count_syllables(word) for word in filtered_words) / word_count
    avg_word_length = sum(len(word) for word in filtered_words) / word_count

    # Personal Pronouns
    personal_pronouns = len([word for word in filtered_words if re.match(r'\b(I|we|my|ours|us)\b', word, re.I)])

    return {
        'URL_ID':data['URL_ID'],
        'URL':df1['URL'],
        'Positive Score': positive_score,
        'Negative Score': negative_score,
        'Polarity Score': polarity_score,
        'Subjectivity Score': subjectivity_score,
        'Avg Sentence Length': avg_sentence_length,
        'Percentage of Complex Words': percentage_complex_words,
        'Fog Index': fog_index,
        'Avg Number of words per Sentence':avg_sentences,
        'Complex Word Count': len(complex_words),
        'Word Count': word_count,
        'Syllable per Word': syllables_per_word,
        'Personal Pronouns': personal_pronouns,
        'Avg Word Length': avg_word_length
        
    }

def count_syllables(word):
    vowels = 'aeiouy'
    word = word.lower()
    syllable_count = 0
    if word[0] in vowels:
        syllable_count += 1
    for i in range(1, len(word)):
        if word[i] in vowels and word[i - 1] not in vowels:
            syllable_count += 1
    if word.endswith('e'):
        syllable_count -= 1
    return max(1, syllable_count)

# Analyze Extracted Articles
output_data = []

for index, row in data.iterrows():
    article_text = row['Article_Text']
    if article_text:
        scores = calculate_scores(article_text)
        scores['URL_ID'] = row['URL_ID']
        scores['URL']=row['URL']
        output_data.append(scores)

# Save Output to Excel
output_df = pd.DataFrame(output_data)
# Define column order
column_order = ['URL_ID', 'URL', 'Positive Score', 'Negative Score', 'Polarity Score',
                'Subjectivity Score', 'Avg Sentence Length', 'Percentage of Complex Words',
                'Fog Index','Avg Number of words per Sentence','Complex Word Count','Word Count','Syllable per Word','Personal Pronouns','Avg Word Length']

# Reorder columns in the DataFrame
output_df = output_df[column_order]

output_df.to_excel('Output_Data_Structure.xlsx', index=False)


[nltk_data] Downloading package punkt to C:\Users\SATEESH
[nltk_data]     REDDY\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\SATEESH
[nltk_data]     REDDY\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


NameError: name 'df1' is not defined

In [ ]:
df=pd.read_excel("Output_Data_Structure.xlsx")

In [ ]:
df

In [ ]:
from openpyxl import Workbook
from openpyxl.styles import Font

# Load the dataset (Input.xlsx)
input_data = pd.read_excel('Output_Data_Structure.xlsx')

# Print the column names to ensure correct column
print(input_data.columns.tolist())

# Ensure 'URL' column exists and remove extra spaces if needed
input_data.columns = input_data.columns.str.strip()  # Remove leading/trailing spaces

# Check if there are any NaN values in the 'URL' column
if input_data['URL'].isna().sum() > 0:
    print(f"Warning: Found {input_data['URL'].isna().sum()} missing URLs")
    # Optional: Fill NaN URLs with a placeholder (or handle them differently)
    input_data['URL'] = input_data['URL'].fillna('No URL')  # You can adjust this as needed

# Create a new workbook and select the active sheet
wb = Workbook()
ws = wb.active

# Add the header row from the original dataframe to the Excel sheet
ws.append(input_data.columns.tolist())

# Define the color for hyperlinks (e.g., blue)
hyperlink_color = "0000FF"  # Blue color code

# Loop through each row of the DataFrame and insert the hyperlinks for the URL column
for index, row in input_data.iterrows():
    row_data = row.tolist()
    
    # Get the URL and make it clickable by applying the HYPERLINK formula
    url = row['URL']  # Assuming 'URL' is the column with the links
    cell = f'=HYPERLINK("{url}", "{url}")'
    row_data[input_data.columns.get_loc('URL')] = cell
    
    # Append the modified row with the clickable URL
    ws.append(row_data)
    
    # Change the font color for the hyperlink (assuming the URL column is at index 1)
    hyperlink_cell = ws.cell(row=index + 2, column=input_data.columns.get_loc('URL') + 1)
    hyperlink_cell.font = Font(color=hyperlink_color)

# Save the new workbook to an Excel file
wb.save('Output_Data_Structure.xlsx')

In [ ]:
df1=pd.read_excel("Output_Data_Structure.xlsx")

In [ ]:
df1